This notebook researches how BitQuery GraphQL API interacts with Uniswap data.

# Imports

In [ ]:
import os
from typing import Any, Dict, List

import pandas as pd
import requests

# Functions

In [ ]:
def run_query(query: str):
    headers = {"X-API-KEY": os.environ["API_KEY"]}
    request = requests.post(
        "https://graphql.bitquery.io/", json={"query": query}, headers=headers
    )
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception(
            "Query failed and return code is {}.      {}".format(
                request.status_code, query
            )
        )

In [ ]:
def json_to_df(data: List[Dict[Any, Any]]) -> pd.DataFrame:
    """
    Transform the data to Dataframe and set the time index.
    """
    df = pd.json_normalize(data, sep="_")
    df = df.set_index("timeInterval_minute")
    return df

# Get data

**1 min data example for WETH/USDT pair**

In [ ]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 20000, asc: "timeInterval.minute"}
      date: {since: "2023-01-01", till: "2023-01-10"}
      baseCurrency: {is: "0xdac17f958d2ee523a2206206994597c13d831ec7"}
      quoteCurrency: {is: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [ ]:
df = json_to_df(result["data"]["ethereum"]["dexTrades"])
df

The query was made at 2023-01-10 10:33:00, so real-time delay is ~3-4 minutes.

**Get historical data Uniswap from november 2018 with no token pairs filter**

In [ ]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 20000, asc: "timeInterval.minute"}
      date: {since: "2018-11-01", till: "2018-12-01"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [ ]:
df = json_to_df(result["data"]["ethereum"]["dexTrades"])
df

**Get bigger: download all swap data for today (2023-01-10)**

In [ ]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 100000, asc: "timeInterval.minute"}
      date: {since: "2023-01-10"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [ ]:
result

It seems like we have a limitation for number of rows by one query, let's try to write the loop and export every 25k rows.

In [ ]:
# Use limit and offset parameters.
query = """query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 25000, offset: %d, asc: "timeInterval.minute"}
      date: {since: "2023-01-10"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""

limit = 25000
offset = 0
data_dfs = []

for i in range(3):
    # Error message above says there are 61k rows, so 3 iterations by 25k would be enough.
    formatted_query = query % offset
    result = run_query(formatted_query)
    df = json_to_df(result["data"]["ethereum"]["dexTrades"])
    data_dfs.append(df)
    offset += 25000

full_data = pd.concat(data_dfs)

In [ ]:
full_data

The query was made at 2023-01-10 10:42:00, so real-time delay is 5 min.